<a href="https://colab.research.google.com/github/gitboku/statistics-practice/blob/master/%E3%83%AD%E3%82%B8%E3%82%B9%E3%83%86%E3%82%A3%E3%83%83%E3%82%AF%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ロジスティック回帰分析とは

目的変数を0から1の範囲でとりたいときに有効な分析方法。
例えば、癌の発生確率など。

## 非線形回帰とは

単回帰分析と重回帰分析は線警戒機。
非線形は直線ではない関係、例えば2上とかルートの関係。
ロジスティック回帰分析以外には多項式回帰分析とかサポートベクトル回帰分析などがある。

## 数式

ロジスティック回帰分析では以下の式を取る。

$$
p = \frac {1} {1+ \exp(-(a_1x_1 + \cdots + a_nx_n + b))}
$$

これを変形すると以下のようになる。

$$
\ln{\frac {p} {1-p}} = a_1x_1 + \cdots + a_nx_n + b
$$

なお、上のほうの式はシグモイド関数とも呼ばれる。

---
また、上記の式をロジットと呼び、$l$で置くことがある。
このロジットに対して指数を取ると、オッズが出てくる。

$$
\exp(l) = \frac {p} {1 - p}
$$

このオッズはある事象が発生する確率と発生しない確率の比になっている。

---

## 偏回帰係数の求め方

ロジスティック回帰分析では単回帰や重回帰のように最小二乗法で求めることはできないらしい。  

参考：　https://qiita.com/karaage0703/items/417934d82ac3c3c5f70e#%E3%83%AD%E3%82%B8%E3%82%B9%E3%83%86%E3%82%A3%E3%83%83%E3%82%AF%E5%9B%9E%E5%B8%B0%E5%88%86%E6%9E%90%E3%81%AE%E8%A7%A3%E3%81%8D%E6%96%B9


よって、勾配降下法などを使用して逐次計算でパラメータ推定をする。

## 勾配降下法について

いくつかの種類がある。

*   最急降下法（Gradient Descent）
*   確率的勾配降下法（Stochastic Gradient Descent）
*   ミニバッチ確率的勾配降下法（Minibatch SGD）

計算量が少ない、オンライン学習が可能といったメリットから、たいていの場合確率的勾配降下法を使用するが、異常値に引っ張られやすい。

ミニバッチ法ではまとまったデータをいくつかの組に分けて確率的勾配降下法を行う。
全データを使用する回数のことをエポック数という。
例えば、データ数が1000で、ミニバッチのサイズが50の時、20回学習を行うと全データを1周するので、20反復が1エポックとなる。

エポック数が10なら10000のデータを使用する。ミニバッチサイズが50なので、200回学習することになる。